# Platform API - B2B Integration Guide

**Last Updated**: 01 FEB 2026
**Version**: 0.8.6.2 (API Version 1.5.0)

Complete workflow for submitting, approving, and accessing geospatial data through the Platform API.

## Workflow Overview

```
0. Validate    POST /api/platform/submit?dry_run=true   Check lineage, get suggested params
       ↓       (or POST /api/platform/validate)
1. Submit      POST /api/platform/submit        Submit raster or vector for processing
       ↓                                        (include previous_version_id for v2+)
2. Poll        GET /api/platform/status/{id}    Monitor until completed
       ↓
3. Approve     POST /api/platform/approve       Set clearance_level (ouo/public), approve dataset
   (or Reject) POST /api/platform/reject        Reject with reason if data fails review
       ↓
4. Access      {TITILER_URL}/stac/...           Query STAC catalog, get tiles
       ↓
5. Revoke      POST /api/platform/revoke        Unapprove (required before unpublish)
       ↓                                        (or use force_approved=true in unpublish)
6. Unpublish   POST /api/platform/unpublish     Delete data (STAC item, table/COG)

Recovery:
   Resubmit    POST /api/platform/resubmit      Clean up failed job and retry
```

## Asset State Model

Assets have **three orthogonal state dimensions**:

### 1. Processing Status (workflow execution)
| Status | Description |
|--------|-------------|
| `pending` | Request received, job not started |
| `processing` | Job running |
| `completed` | Job finished successfully |
| `failed` | Job failed (may retry via resubmit) |

### 2. Approval State (data quality review)
| State | Description | Next Actions |
|-------|-------------|--------------|
| `pending_review` | Job completed, awaiting approval | Approve or Reject |
| `approved` | Data quality confirmed | Access data, Revoke, or Unpublish |
| `rejected` | Rejected with reason | Resubmit after fixes |

### 3. Clearance State (access control)
| Level | Description |
|-------|-------------|
| `uncleared` | Default - same access as OUO, awaiting confirmation |
| `ouo` | Official Use Only (internal access) |
| `public` | Triggers ADF export to external zone |

**Note:** `uncleared` and `ouo` have identical access behavior. The distinction is for B2B workflow confirmation.

## Approval Record Status (Separate from Asset)

The `DatasetApproval` record tracks the approval workflow:

| Status | Transition From | Description |
|--------|-----------------|-------------|
| `pending` | (initial) | Awaiting reviewer |
| `approved` | `pending` | Approved for access |
| `rejected` | `pending` | Rejected with reason |
| `revoked` | `approved` | Unapproved (before unpublish) |

---

## Setup & Configuration

### Before You Start

1. **Verify your test files exist** in Azure Storage:
   - Vector: GeoJSON, Shapefile, GeoPackage, FlatGeobuf
   - Raster: GeoTIFF (.tif)

2. **Update the DDH identifiers** (dataset_id, resource_id, version_id) for your test data

---

### Configuration Steps

**Step 1: Verify Environment URLs** (lines 11-17 below)

The QA environment URLs are pre-configured:
- Platform API: `https://gddatahubetlqa-qa.ocappsaseqa2.appserviceenvironment.net`
- Service Layer: `https://gddatahubpyfeqa.ocappsaseqa2.appserviceenvironment.net`

**Step 2: Verify Storage Container** (line 23)

QA bronze container: `gddathub-bronze-qa`

**Step 3: Configure Test Data** (lines 29-46)

Update the identifiers to match YOUR test files:

```python
TEST_VECTOR = {
    "container_name": BRONZE_CONTAINER,
    "file_name": "roads.gpkg",             # ← Your file path
    "dataset_id": "my-project",            # ← Replace with your dataset ID
    "resource_id": "roads",                # ← Replace with your resource ID
    "version_id": "v1"                     # ← Replace with your version ID
}
```

---

### Quick Verification

After running the setup cell, you should see:
```
============================================================
PLATFORM API - QA ENVIRONMENT
============================================================

Endpoints:
  Platform API:     https://gddatahubetlqa-qa...
  Service Layer:    https://gddatahubpyfeqa...
...
```

---

### Run This Cell ↓

In [ ]:
import requests
import json
import time
from IPython.display import Markdown, display

# =============================================================================
# STEP 1: ENVIRONMENT URLS (QA)
# =============================================================================

ENVIRONMENT = "QA"

# Platform API - where you submit jobs and check status
BASE_URL = "https://gddatahubetlqa-qa.ocappsaseqa2.appserviceenvironment.net"

# Service Layer - where you access published data (STAC, tiles, features)
TITILER_URL = "https://gddatahubpyfeqa.ocappsaseqa2.appserviceenvironment.net"

# =============================================================================
# STEP 2: STORAGE CONTAINER
# =============================================================================

BRONZE_CONTAINER = "gddathub-bronze-qa"

# =============================================================================
# STEP 3: TEST DATA FILES
# =============================================================================
# Update the identifier placeholders to match your test data

# Vector test file - GeoPackage
TEST_VECTOR = {
    "container_name": BRONZE_CONTAINER,
    "file_name": "roads.gpkg",
    "dataset_id": "your-dataset-id",      # ← Replace with your dataset ID
    "resource_id": "your-resource-id",    # ← Replace with your resource ID
    "version_id": "v1"                    # ← Replace with your version ID
}

# Raster test file - GeoTIFF
TEST_RASTER = {
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif",
    "dataset_id": "your-dataset-id",      # ← Replace with your dataset ID
    "resource_id": "your-resource-id",    # ← Replace with your resource ID
    "version_id": "v1"                    # ← Replace with your version ID
}

# =============================================================================
# HELPER FUNCTIONS (Don't modify)
# =============================================================================

def api_call(method, endpoint, data=None, params=None, timeout=30):
    """Make API call and return formatted response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    
    print(f"\n{'='*60}")
    print(f"{method} {endpoint}")
    print(f"{'='*60}")
    
    if data:
        print(f"\nRequest Body:")
        print(json.dumps(data, indent=2))
    
    try:
        if method == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print(f"\nStatus: {response.status_code}")
        
        try:
            result = response.json()
            print(f"\nResponse:")
            print(json.dumps(result, indent=2, default=str))
            return result
        except:
            print(f"\nResponse (text): {response.text[:500]}")
            return {"error": response.text, "status_code": response.status_code}
            
    except requests.exceptions.Timeout:
        print(f"\nRequest timed out (timeout={timeout}s)")
        return None
    except requests.exceptions.ConnectionError:
        print(f"\n❌ CONNECTION ERROR - Check your network connection")
        print(f"   URL: {url}")
        return None
    except Exception as e:
        print(f"\nError: {e}")
        return None


def poll_status(request_id, max_polls=30, poll_interval=5):
    """Poll platform status until job completes."""
    print(f"\n{'='*60}")
    print(f"Polling request: {request_id}")
    print(f"{'='*60}")
    
    for i in range(max_polls):
        try:
            result = requests.get(
                f"{BASE_URL}/api/platform/status/{request_id}", 
                timeout=30
            ).json()
        except:
            print(f"  [{i+1}/{max_polls}] Connection error - retrying...")
            time.sleep(poll_interval)
            continue
        
        job_status = result.get("job_status", "unknown")
        job_stage = result.get("job_stage", "?")
        job_id = result.get("job_id", "N/A")[:16] if result.get("job_id") else "N/A"
        
        print(f"  [{i+1}/{max_polls}] Status: {job_status}, Stage: {job_stage}, Job: {job_id}...")
        
        if job_status in ["completed", "failed"]:
            print(f"\nFinal Result:")
            print(json.dumps(result, indent=2, default=str))
            return result
        
        time.sleep(poll_interval)
    
    print(f"\nPolling timeout after {max_polls * poll_interval}s")
    return result


# =============================================================================
# CONFIGURATION CHECK
# =============================================================================

print("=" * 60)
print(f"PLATFORM API - {ENVIRONMENT} ENVIRONMENT")
print("=" * 60)

print(f"\nEndpoints:")
print(f"  Platform API:     {BASE_URL}")
print(f"  Service Layer:    {TITILER_URL}")
print(f"  API Docs:         {TITILER_URL}/docs")
print(f"\nStorage:")
print(f"  Bronze Container: {BRONZE_CONTAINER}")
print(f"\nTest Data:")
print(f"  Vector: {TEST_VECTOR['file_name']}")
print(f"  Raster: {TEST_RASTER['file_name']}")
print("=" * 60)

---
# Part 1: Platform Health

Check if the platform is ready to accept jobs.

**Endpoint:** `GET /api/platform/health`

Returns simplified health status for external applications (no internal details exposed).

In [ ]:
# Platform Health Check
result = api_call("GET", "/api/platform/health")

if result:
    print(f"\n{'='*60}")
    print("PLATFORM STATUS")
    print(f"{'='*60}")
    print(f"Ready for Jobs: {result.get('ready_for_jobs', 'unknown')}")
    print(f"Queue Backlog:  {result.get('queue_backlog', 'unknown')}")
    print(f"Version:        {result.get('version', 'unknown')}")

---
# Part 2: Complete Platform Workflow

Full lifecycle: Validate → Submit → Poll → Approve → Unpublish

## 2.1 Pre-flight Validation & Version Lineage

Validate request before submitting and check version lineage state.

**Two equivalent options:**
- `POST /api/platform/validate` - Standalone validation endpoint
- `POST /api/platform/submit?dry_run=true` - Submit with dry_run parameter

Both return identical responses including **lineage state** for version tracking.

### Parameters

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `dataset_id` | body | Yes | DDH dataset identifier |
| `resource_id` | body | Yes | DDH resource identifier |
| `version_id` | body | Yes | Version identifier (e.g., `v2.0`) |
| `container_name` | body | Yes | Storage container |
| `file_name` | body | Yes | Path to file |
| `previous_version_id` | body | For v2+ | Must match current latest version |

### Response includes:

- `valid`: Whether submission would succeed
- `lineage_state`: Current version chain info
- `suggested_params`: Recommended parameters (e.g., `previous_version_id`)
- `warnings`: Validation messages

In [ ]:
# =============================================================================
# PRE-FLIGHT VALIDATION WITH DRY_RUN
# =============================================================================
# Uses POST /api/platform/submit?dry_run=true to check lineage and validate
# This returns the same response as POST /api/platform/validate

test_file = TEST_RASTER  # or TEST_VECTOR

validate_request = {
    "dataset_id": test_file["dataset_id"],
    "resource_id": test_file["resource_id"],
    "version_id": test_file["version_id"],
    "container_name": test_file["container_name"],
    "file_name": test_file["file_name"]
    # Note: previous_version_id NOT included - let validation tell us what to set
}

# Use dry_run=true query parameter
result = api_call("POST", "/api/platform/submit?dry_run=true", validate_request)

if result:
    is_valid = result.get("valid", False)
    lineage = result.get("lineage_state", {})
    suggested = result.get("suggested_params", {})
    warnings = result.get("warnings", [])
    
    print(f"\n{'='*60}")
    print("VALIDATION RESULT")
    print(f"{'='*60}")
    print(f"Valid: {is_valid}")
    print(f"Would create job type: {result.get('would_create_job_type', 'N/A')}")
    
    if lineage:
        print(f"\nLineage State:")
        print(f"  Lineage ID: {lineage.get('lineage_id', 'N/A')[:16]}...")
        print(f"  Lineage exists: {lineage.get('lineage_exists', False)}")
        if lineage.get("current_latest"):
            latest = lineage["current_latest"]
            print(f"  Current latest version: {latest.get('version_id')}")
            print(f"  Version ordinal: {latest.get('version_ordinal')}")
    
    if suggested:
        print(f"\nSuggested Parameters:")
        for key, value in suggested.items():
            print(f"  {key}: {value}")
    
    if warnings:
        print(f"\nWarnings:")
        for w in warnings:
            print(f"  ⚠️ {w}")
    
    if is_valid:
        print(f"\n✅ Ready to submit")
    else:
        print(f"\n❌ Not valid - check warnings/errors above")

## 2.2 Submit Data

Submit raster or vector data for processing.

**Endpoint:** `POST /api/platform/submit`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `dataset_id` | body | Yes | Top-level identifier (e.g., `aerial-imagery`) |
| `resource_id` | body | Yes | Resource identifier (e.g., `site-alpha`) |
| `version_id` | body | Yes | Version identifier (e.g., `v1.0`) |
| `container_name` | body | Yes | Source container with raw file |
| `file_name` | body | Yes | Blob path to file |
| `previous_version_id` | body | **For v2+** | Must match current latest version |
| `title` | body | No | Human-readable name |
| `access_level` | body | No | `ouo` (default) or `public` |

### Version Lineage Rules

| Scenario | `previous_version_id` | Result |
|----------|----------------------|--------|
| First version (v1.0) | Omit | ✅ Creates lineage |
| New version (v2.0) after v1.0 | `"v1.0"` | ✅ Advances lineage |
| New version without prev | Omit | ❌ Error: specify previous |
| Wrong previous_version_id | `"v0.5"` | ❌ Error: doesn't match latest |

**Note:** Data type is auto-detected from file extension.

## 2.1b Version Lineage Example: v1.0 → v2.0

This example demonstrates submitting a new version when a previous version exists.

**Workflow:**
1. Validate to check lineage state (discovers v1.0 exists)
2. Submit v2.0 with `previous_version_id="v1.0"`

In [ ]:
# =============================================================================
# VERSION LINEAGE EXAMPLE: Submit v2.0 after v1.0
# =============================================================================
# This example shows how to properly advance a version

# Step 1: Define v2.0 submission (same dataset/resource, new version)
v2_request = {
    "dataset_id": TEST_RASTER["dataset_id"],
    "resource_id": TEST_RASTER["resource_id"],
    "version_id": "v2.0",  # New version
    "container_name": TEST_RASTER["container_name"],
    "file_name": TEST_RASTER["file_name"]  # Could be different file
}

# Step 2: Validate first to check lineage
print("Step 1: Checking lineage state...")
result = api_call("POST", "/api/platform/submit?dry_run=true", v2_request)

if result:
    is_valid = result.get("valid", False)
    suggested = result.get("suggested_params", {})
    
    if is_valid:
        print(f"\n✅ Valid! Ready to submit v2.0")
    else:
        # Get suggested previous_version_id from response
        prev_version = suggested.get("previous_version_id")
        if prev_version:
            print(f"\n⚠️ Lineage exists - need to specify previous_version_id")
            print(f"   Suggested: previous_version_id='{prev_version}'")
            
            # Step 3: Add previous_version_id and submit
            print(f"\nStep 2: Adding previous_version_id and submitting...")
            v2_request["previous_version_id"] = prev_version
            
            # Uncomment to actually submit:
            # result = api_call("POST", "/api/platform/submit", v2_request)
            # if result and result.get("request_id"):
            #     print(f"✅ v2.0 submitted! Request ID: {result['request_id']}")
            
            print(f"\n   To submit, uncomment the code above and run again")
        else:
            print(f"\n❌ Validation failed: {result.get('warnings', [])}")

In [ ]:
# =============================================================================
# SUBMIT DATA - Choose ONE: Raster OR Vector
# =============================================================================

# Option A: Submit RASTER
submit_request = {
    "dataset_id": TEST_RASTER["dataset_id"],
    "resource_id": TEST_RASTER["resource_id"],
    "version_id": TEST_RASTER["version_id"],
    "container_name": TEST_RASTER["container_name"],
    "file_name": TEST_RASTER["file_name"],
    "access_level": "ouo"
    # For subsequent versions, add:
    # "previous_version_id": "v1.0"  # Must match current latest
}

# Option B: Submit VECTOR (uncomment to use instead)
# submit_request = {
#     "dataset_id": TEST_VECTOR["dataset_id"],
#     "resource_id": TEST_VECTOR["resource_id"],
#     "version_id": TEST_VECTOR["version_id"],
#     "container_name": TEST_VECTOR["container_name"],
#     "file_name": TEST_VECTOR["file_name"],
#     "access_level": "ouo"
#     # For subsequent versions, add:
#     # "previous_version_id": "v1.0"  # Must match current latest
# }

# =============================================================================

result = api_call("POST", "/api/platform/submit", submit_request)

# Store request_id for subsequent steps
REQUEST_ID = result.get("request_id") if result else None

if REQUEST_ID:
    print(f"\n✅ Submitted successfully")
    print(f"   Request ID: {REQUEST_ID}")
    print(f"   Job ID:     {result.get('job_id', 'N/A')[:16]}...")
    print(f"   Job Type:   {result.get('job_type', 'N/A')}")
    print(f"\n   Use REQUEST_ID for all subsequent operations")
else:
    # Check if this was a validation error (lineage issue)
    if result and result.get("error_type") == "ValidationError":
        print(f"\n❌ Validation failed: {result.get('error')}")
        print(f"\n   Hint: Run the validation cell first to check lineage state")
    else:
        print(f"\n❌ Submit failed")

## 2.3 Poll Status

Monitor job progress using the request ID.

**Endpoint:** `GET /api/platform/status/{request_id}`

Returns job status, stage progress, and result data when completed.

In [ ]:
# Poll Status by Request ID
if REQUEST_ID:
    result = poll_status(REQUEST_ID, max_polls=30, poll_interval=5)
    
    if result and result.get("job_status") == "completed":
        print(f"\n✅ Job completed successfully")
        # Store for approval step
        JOB_ID = result.get("job_id")
    elif result and result.get("job_status") == "failed":
        print(f"\n❌ Job failed")
        JOB_ID = None
else:
    print("No REQUEST_ID - run Submit cell first")
    JOB_ID = None

## 2.4 Approve or Reject Dataset

After a job completes, approve it for publication or reject if it fails review.

### Approve

**Endpoint:** `POST /api/platform/approve`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `job_id` | body | Yes* | Job ID from completed job |
| `clearance_level` | body | **Yes** | `ouo` or `public` |
| `reviewer` | body | Yes | Email of approver |
| `notes` | body | No | Approval notes |

*Can also use `request_id` instead of `job_id`

### Reject (Alternative - No Live Example)

If the data fails review, reject it instead:

**Endpoint:** `POST /api/platform/reject`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `job_id` | body | Yes* | Job ID from completed job |
| `reviewer` | body | Yes | Email of reviewer |
| `reason` | body | **Yes** | Why rejected (audit trail) |

```python
# Example reject request (not executed)
reject_request = {
    "job_id": JOB_ID,
    "reviewer": "qa-tester@example.com",
    "reason": "Data quality issue: missing CRS metadata"
}
# result = api_call("POST", "/api/platform/reject", reject_request)
```

Rejected datasets can be resubmitted after fixes via `POST /api/platform/resubmit`.

In [ ]:
# Approve Dataset
if JOB_ID:
    approve_request = {
        "job_id": JOB_ID,
        "clearance_level": "ouo",  # REQUIRED: "ouo" or "public"
        "reviewer": "qa-tester@example.com",
        "notes": "QA test approval"
    }
    
    result = api_call("POST", "/api/platform/approve", approve_request)
    
    if result and result.get("success"):
        print(f"\n✅ Dataset approved")
        print(f"   Approval ID: {result.get('approval_id', 'N/A')}")
        print(f"   Status: {result.get('status', 'N/A')}")
        print(f"   Clearance: {result.get('clearance_level', 'N/A')}")
    else:
        print(f"\n❌ Approval failed")
else:
    print("No JOB_ID - run Poll cell first (job must be completed)")

## 2.5 Revoke and Unpublish Dataset

To remove an **approved** dataset, you must either revoke first OR use `force_approved=true`.

### Option 1: Two-Step (Explicit Audit Trail)

```
Step 1: POST /api/platform/revoke     → Unapproves the dataset
Step 2: POST /api/platform/unpublish  → Deletes the data
```

### Option 2: Force Unpublish (Convenience)

```
POST /api/platform/unpublish with force_approved=true
```
This auto-revokes approved data before deleting.

---

### Step 1: Revoke Approval (Optional)

**Endpoint:** `POST /api/platform/revoke`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `approval_id` | body | Yes* | Approval ID |
| `revoker` | body | Yes | Email of person revoking |
| `reason` | body | **Yes** | Why revoking (audit trail) |

*Can also use `stac_item_id` or `job_id`

### Step 2: Unpublish Data

**Endpoint:** `POST /api/platform/unpublish`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `dataset_id` | body | Yes* | Original dataset_id |
| `resource_id` | body | Yes* | Original resource_id |
| `version_id` | body | Yes* | Original version_id |
| `dry_run` | body | No | `true` (default) = preview only |
| `force_approved` | body | No | `true` = auto-revoke approved data |

*Can also use `request_id` or `job_id` instead of DDH identifiers.

**Safety:** Always defaults to `dry_run=true`. Set to `false` to actually delete.

In [ ]:
# =============================================================================
# STEP 1: REVOKE APPROVAL (Required before unpublish)
# =============================================================================
# This unapproves the dataset and creates an audit trail
# Must be called before unpublish for approved datasets

if JOB_ID:
    revoke_request = {
        "job_id": JOB_ID,
        "revoker": "qa-tester@example.com",
        "reason": "QA test cleanup - removing test data"  # Required for audit
    }
    
    result = api_call("POST", "/api/platform/revoke", revoke_request)
    
    if result and result.get("success"):
        print(f"\n✅ Approval revoked")
        print(f"   Approval ID: {result.get('approval_id', 'N/A')}")
        print(f"   Status: {result.get('status', 'N/A')}")
        print(f"   Asset deleted: {result.get('asset_deleted', 'N/A')}")
        if result.get("warning"):
            print(f"   ⚠️  {result.get('warning')}")
        print(f"\n   ➡️  Now run the Unpublish cell below to delete data")
    else:
        error = result.get('error', 'Unknown error') if result else 'No response'
        print(f"\n❌ Revoke failed: {error}")
        if "not found" in str(error).lower():
            print("   Hint: Check that the job was approved first")
        elif "approved" in str(error).lower():
            print("   Hint: Only approved datasets can be revoked")
else:
    print("No JOB_ID available")
    print("\nTo revoke manually, use one of these options:")
    print("""
    # Option 1: By Job ID
    revoke_request = {
        "job_id": "your-job-id-here",
        "revoker": "your-email@example.com",
        "reason": "Reason for revocation"
    }
    
    # Option 2: By Approval ID
    revoke_request = {
        "approval_id": "apr-xxxxx",
        "revoker": "your-email@example.com", 
        "reason": "Reason for revocation"
    }
    
    # Option 3: By Request ID
    revoke_request = {
        "request_id": "your-request-id",
        "revoker": "your-email@example.com",
        "reason": "Reason for revocation"
    }
    """)

In [ ]:
# =============================================================================
# STEP 2: UNPUBLISH DATA (Run after revoking)
# =============================================================================
# Deletes the actual data: STAC item, PostGIS table (vector), or COG files (raster)
# Only run this after successfully revoking the approval

test_file = TEST_RASTER  # or TEST_VECTOR

unpublish_request = {
    "dataset_id": test_file["dataset_id"],
    "resource_id": test_file["resource_id"],
    "version_id": test_file["version_id"],
    "dry_run": True  # SAFETY: Set to False to actually delete
}

result = api_call("POST", "/api/platform/unpublish", unpublish_request)

if result:
    if result.get("dry_run"):
        print(f"\n🔍 DRY RUN - No changes made")
        print(f"   Would delete: {result.get('would_delete', {})}")
        print(f"\n   Set dry_run=False to actually delete")
    else:
        print(f"\n✅ Dataset unpublished")
        print(f"   Deleted: {result.get('deleted', {})}")
else:
    print(f"\n❌ Unpublish failed")

---
# Part 3: Catalog Discovery

Query published assets and STAC items through the Platform API.

## Catalog Endpoints

| Endpoint | Purpose |
|----------|---------|
| `GET /api/platform/catalog/lookup` | Lookup by DDH IDs (dataset/resource/version) |
| `GET /api/platform/catalog/assets` | List all assets with filters |
| `GET /api/platform/catalog/item/{col}/{item}` | Get full STAC item |
| `GET /api/platform/catalog/assets/{col}/{item}` | Get asset URLs with TiTiler links |
| `GET /api/platform/catalog/dataset/{id}` | List all items for a dataset |

## 3.1 List Assets

**Endpoint:** `GET /api/platform/catalog/assets`

| Parameter | Type | Description |
|-----------|------|-------------|
| `limit` | query | Max results (default: 100) |
| `approval_state` | query | Filter: `pending_review`, `approved`, `rejected` |
| `clearance_state` | query | Filter: `uncleared`, `ouo`, `public` |

In [ ]:
# List Assets
result = api_call("GET", "/api/platform/catalog/assets", params={"limit": 10})

if result and result.get("assets"):
    assets = result["assets"]
    print(f"\nFound {len(assets)} assets:")
    for asset in assets[:5]:
        print(f"  - {asset.get('dataset_id')}/{asset.get('resource_id')}/{asset.get('version_id')}")
        print(f"    Approval: {asset.get('approval_state')}, Clearance: {asset.get('clearance_state')}")

## 3.2 Lookup by DDH Identifiers

**Endpoint:** `GET /api/platform/catalog/lookup`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `dataset_id` | query | Yes | Dataset identifier |
| `resource_id` | query | Yes | Resource identifier |
| `version_id` | query | No | Specific version (omit for latest) |

In [ ]:
# Lookup Asset by DDH Identifiers
test_file = TEST_RASTER  # or TEST_VECTOR

params = {
    "dataset_id": test_file["dataset_id"],
    "resource_id": test_file["resource_id"],
    "version_id": test_file["version_id"]
}

result = api_call("GET", "/api/platform/catalog/lookup", params=params)

if result and result.get("found"):
    print(f"\n✅ Asset found")
    print(f"   STAC Item: {result.get('stac_item_id', 'N/A')}")
    print(f"   Collection: {result.get('stac_collection_id', 'N/A')}")
    print(f"   Approval: {result.get('approval_state', 'N/A')}")
    print(f"   Clearance: {result.get('clearance_state', 'N/A')}")
else:
    print(f"\n❌ Asset not found")

## 3.3 Get STAC Item Details

**Endpoint:** `GET /api/platform/catalog/item/{collection_id}/{item_id}`

Returns the full STAC item as GeoJSON Feature.

In [ ]:
# Get STAC Item Details
# Requires knowing collection_id and item_id from lookup or asset list

# Replace with your actual IDs (from lookup results)
COLLECTION_ID = "your-dataset-id"  # Usually same as dataset_id
ITEM_ID = "your-dataset-id-resource-id-v1"  # Format: {dataset}-{resource}-{version}

result = api_call("GET", f"/api/platform/catalog/item/{COLLECTION_ID}/{ITEM_ID}")

if result and result.get("type") == "Feature":
    props = result.get("properties", {})
    print(f"\n✅ STAC Item retrieved")
    print(f"   ID: {result.get('id')}")
    print(f"   Collection: {result.get('collection')}")
    print(f"   Datetime: {props.get('datetime', 'N/A')}")
    print(f"   BBox: {result.get('bbox', 'N/A')}")
    assets = result.get("assets", {})
    print(f"   Assets: {list(assets.keys())}")
elif result:
    print(f"\n❌ Item not found or error: {result.get('error', 'Unknown')}")

## 3.4 Get Asset URLs with TiTiler

**Endpoint:** `GET /api/platform/catalog/assets/{collection_id}/{item_id}`

Returns asset URLs with pre-built TiTiler tile/preview links.

| Parameter | Type | Description |
|-----------|------|-------------|
| `include_titiler` | query | `true` = include TiTiler URLs for rasters |

In [ ]:
# Get Asset URLs with TiTiler Links
# Useful for raster data - provides ready-to-use tile URLs

# Replace with your actual IDs
COLLECTION_ID = "your-dataset-id"
ITEM_ID = "your-dataset-id-resource-id-v1"

result = api_call(
    "GET", 
    f"/api/platform/catalog/assets/{COLLECTION_ID}/{ITEM_ID}",
    params={"include_titiler": "true"}
)

if result and result.get("assets"):
    print(f"\n✅ Asset URLs retrieved")
    print(f"   Item: {result.get('item_id')}")
    print(f"   BBox: {result.get('bbox', 'N/A')}")
    
    # Show asset links
    assets = result.get("assets", {})
    for name, asset in assets.items():
        print(f"\n   Asset '{name}':")
        print(f"     href: {asset.get('href', 'N/A')[:60]}...")
        print(f"     type: {asset.get('type', 'N/A')}")
    
    # Show TiTiler links (for rasters)
    titiler = result.get("titiler", {})
    if titiler:
        print(f"\n   TiTiler Links:")
        print(f"     preview: {titiler.get('preview', 'N/A')[:60]}...")
        print(f"     tiles:   {titiler.get('tiles', 'N/A')[:60]}...")
elif result:
    print(f"\n❌ Error: {result.get('error', 'Unknown')}")

## 3.5 List Items for Dataset

**Endpoint:** `GET /api/platform/catalog/dataset/{dataset_id}`

Returns all STAC items for a given dataset (across all resources and versions).

| Parameter | Type | Description |
|-----------|------|-------------|
| `limit` | query | Max results (default: 50) |

In [ ]:
# List All Items for a Dataset
DATASET_ID = TEST_RASTER["dataset_id"]  # or TEST_VECTOR["dataset_id"]

result = api_call(
    "GET", 
    f"/api/platform/catalog/dataset/{DATASET_ID}",
    params={"limit": 50}
)

if result and result.get("items"):
    items = result["items"]
    print(f"\n✅ Found {result.get('count', len(items))} items for dataset '{DATASET_ID}'")
    
    for item in items[:10]:  # Show first 10
        print(f"\n   {item.get('item_id')}:")
        print(f"     Resource: {item.get('resource_id')}")
        print(f"     Version:  {item.get('version_id')}")
        print(f"     Datetime: {item.get('datetime', 'N/A')}")
elif result:
    if result.get("count") == 0:
        print(f"\n⚠️ No items found for dataset '{DATASET_ID}'")
    else:
        print(f"\n❌ Error: {result.get('error', 'Unknown')}")

---
# Part 3.5: Platform Registry

Query supported B2B platforms and their identifier requirements.

## Endpoints

| Endpoint | Purpose |
|----------|---------|
| `GET /api/platforms` | List all supported platforms |
| `GET /api/platforms/{id}` | Get specific platform details |

In [ ]:
# List Supported Platforms
result = api_call("GET", "/api/platforms")

if result and result.get("platforms"):
    platforms = result["platforms"]
    print(f"\n✅ Found {result.get('count', len(platforms))} platform(s)")
    
    for p in platforms:
        print(f"\n   Platform: {p.get('platform_id')}")
        print(f"     Name: {p.get('display_name')}")
        print(f"     Required refs: {p.get('required_refs')}")
        print(f"     Optional refs: {p.get('optional_refs')}")
        print(f"     Active: {p.get('is_active')}")
elif result:
    print(f"\n❌ Error: {result.get('error', 'Unknown')}")

---
# Part 4: Data Access

Access published data through the consolidated service layer.

**Base URL:** `TITILER_URL`

**Full API Documentation:** `{TITILER_URL}/docs`

## Service Layer Endpoints

| Path Prefix | Service | Purpose |
|-------------|---------|---------- |
| `/stac/` | STAC API | Catalog search, collections, items |
| `/vector/` | TiPG (OGC Features) | Vector collections, features, vector tiles |
| `/cog/` | TiTiler COG | Raster tiles, info, statistics |
| `/searches/` | Mosaic Search | Dynamic mosaic tiles from STAC searches |

## Quick Reference

```python
# STAC Catalog
f"{TITILER_URL}/stac/collections"                    # List collections
f"{TITILER_URL}/stac/search"                         # Search items (POST)
f"{TITILER_URL}/stac/collections/{id}/items/{item}"  # Get item

# Vector Features (OGC API)
f"{TITILER_URL}/vector/collections"                  # List vector collections
f"{TITILER_URL}/vector/collections/{id}/items"       # Get features
f"{TITILER_URL}/vector/collections/{id}/tiles/..."   # Vector tiles (MVT)

# Raster Tiles (COG)
f"{TITILER_URL}/cog/info?url={cog_url}"              # COG metadata
f"{TITILER_URL}/cog/tiles/{z}/{x}/{y}?url={cog_url}" # XYZ tiles
f"{TITILER_URL}/cog/statistics?url={cog_url}"        # Band statistics
```

In [ ]:
# Data Access Quick Links
print(f"{'='*60}")
print("DATA ACCESS ENDPOINTS")
print(f"{'='*60}")
print(f"\nAPI Documentation:")
print(f"  {TITILER_URL}/docs")
print(f"\nSTAC Catalog:")
print(f"  {TITILER_URL}/stac/collections")
print(f"  {TITILER_URL}/stac/search")
print(f"\nVector Features:")
print(f"  {TITILER_URL}/vector/collections")
print(f"\nHealth Checks:")
print(f"  {TITILER_URL}/health")
print(f"  {TITILER_URL}/stac/_mgmt/ping")

---
# Appendix: Job Recovery

## A.1 Resubmit Failed Job

Clean up and resubmit a failed job with the same parameters. Useful for retrying jobs that failed due to transient errors.

**Endpoint:** `POST /api/platform/resubmit`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `dataset_id` | body | - | DDH dataset identifier (Option 1) |
| `resource_id` | body | - | DDH resource identifier (Option 1) |
| `version_id` | body | - | DDH version identifier (Option 1) |
| `request_id` | body | - | Platform request ID (Option 2) |
| `job_id` | body | - | CoreMachine job ID (Option 3) |
| `dry_run` | body | `false` | Preview cleanup plan without executing |
| `delete_blobs` | body | `false` | Also delete COG files from storage |
| `force` | body | `false` | Resubmit even if job is processing |

**Identifier Options (choose one):**
1. DDH Identifiers: `dataset_id` + `resource_id` + `version_id` (preferred)
2. Request ID: `request_id` from original submit
3. Job ID: `job_id` from CoreMachine

**Response includes:**
- `original_job_id`: The job that was cleaned up
- `new_job_id`: The newly submitted job
- `cleanup_summary`: What was deleted (tasks, STAC items, tables)
- `monitor_url`: URL to poll for new job status

In [ ]:
# =============================================================================
# RESUBMIT FAILED JOB
# =============================================================================
# Choose ONE identifier option below

# Option 1: By DDH Identifiers (Preferred)
# Use the same identifiers from the original submit
test_file = TEST_RASTER  # or TEST_VECTOR

resubmit_request = {
    "dataset_id": test_file["dataset_id"],
    "resource_id": test_file["resource_id"],
    "version_id": test_file["version_id"],
    "dry_run": True,        # Preview first
    "delete_blobs": False   # Keep COG files
}

# Option 2: By Request ID (uncomment to use)
# resubmit_request = {
#     "request_id": REQUEST_ID,  # From original submit
#     "dry_run": True
# }

# Option 3: By Job ID (uncomment to use)
# resubmit_request = {
#     "job_id": "YOUR_JOB_ID_HERE",
#     "dry_run": True
# }

# =============================================================================

result = api_call("POST", "/api/platform/resubmit", resubmit_request)

if result and result.get("success"):
    if result.get("dry_run"):
        print(f"\n🔍 DRY RUN - Cleanup Plan:")
        plan = result.get("cleanup_plan", {})
        print(f"   Tasks to delete: {plan.get('tasks_to_delete', 0)}")
        print(f"   STAC items: {plan.get('stac_items_to_delete', [])}")
        print(f"   Tables to drop: {plan.get('tables_to_drop', [])}")
        print(f"\n   Set dry_run=False to execute resubmit")
    else:
        print(f"\n✅ Job resubmitted successfully")
        print(f"   Original Job: {result.get('original_job_id', 'N/A')[:16]}...")
        print(f"   New Job:      {result.get('new_job_id', 'N/A')[:16]}...")
        print(f"   Monitor URL:  {result.get('monitor_url', 'N/A')}")
        
        # Store new job for polling
        NEW_JOB_ID = result.get("new_job_id")
elif result:
    print(f"\n❌ Resubmit failed: {result.get('error', 'Unknown error')}")

---

## Additional Resources

- **Platform API Health:** `GET /api/platform/health`
- **Platform Failures:** `GET /api/platform/failures?hours=24`
- **Data Lineage:** `GET /api/platform/lineage/{request_id}`
- **Pending Approvals:** `GET /api/platform/approvals`
- **Reject Dataset:** `POST /api/platform/reject`
- **Revoke Approval:** `POST /api/platform/revoke`
- **Resubmit Failed Job:** `POST /api/platform/resubmit`
- **Platform Registry:** `GET /api/platforms`

## Version Lineage Quick Reference

| Scenario | `previous_version_id` | Result |
|----------|----------------------|--------|
| First version | Omit | ✅ Creates new lineage |
| New version | Set to current latest | ✅ Advances lineage |
| New version | Omitted when lineage exists | ❌ Error with suggestion |
| Wrong previous | Doesn't match latest | ❌ Error with correct value |

**Tip:** Always run `POST /api/platform/submit?dry_run=true` first to check lineage state.

## State Reference

### Asset State Dimensions (GeospatialAsset)

| Dimension | Values | Set By |
|-----------|--------|--------|
| Processing Status | `pending`, `processing`, `completed`, `failed` | System (automatic) |
| Approval State | `pending_review`, `approved`, `rejected` | Reviewer (approve/reject) |
| Clearance State | `uncleared`, `ouo`, `public` | Reviewer (during approval) |

### Approval Record Status (DatasetApproval)

| Status | Description |
|--------|-------------|
| `pending` | Awaiting reviewer decision |
| `approved` | Approved for access |
| `rejected` | Rejected with reason |
| `revoked` | Previously approved, now unapproved (for unpublish) |

**Note:** `revoked` is only reached from `approved` via `/api/platform/revoke` or `force_approved=true` during unpublish.

---

*Generated for Platform API v0.8.6.2 (API Version 1.5.0)*
*Last Updated: 01 FEB 2026*